In [134]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score

from scipy.stats import entropy

# import some data to play with
iris = datasets.load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# def change_weights(X_train,y_train,X_test,y_test,weights):

In [135]:
#Check if the entropy function works
pk = np.array([1/5, 2/5, 2/5])  # fair coin
H = entropy(pk)
print(H)
#H=-0.2*(np.log(0.2))-0.4*(np.log(0.4))-0.4*(np.log(0.4))
#print(H)

1.0549201679861442


In [136]:

def calculate_probabilities(list_labels, uniq_labels):
    '''
    Author: Sara Nassar 
    this function calculates the probabilities of each label in the list of labels
    it is calculated by number of labels in class A/all labels
    number of labels in class B/all labels
    and so on
    '''
        
    # Create a dictionary to store the counts of each label
    counts = {}

    # Loop through the labels in list_labels
    for label in list_labels:
        # If the label is not in the dictionary, add it and set its count to 1
        if label not in counts:
            counts[label] = 1
        # If the label is already in the dictionary, increment its count by 1
        else:
            counts[label] += 1
    
    # Finding the unique labels
    uniq = np.unique(uniq_labels)

    # Calculating the total number of labels
    total = len(list_labels)

    # Calculating the probabilities of each label
    probabilities = [counts[label] / total for label in uniq]

    return probabilities

In [137]:
# Test your function
list_labels=[1,2,0,1,2,0]
uniq_labels=[0,1,2]
print(calculate_probabilities(list_labels,uniq_labels))

[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]


In [138]:
'''
Author: Sara Nassar 
list_probas is the list of probabiities
the formula for entropy is
sum(-proba*log(proba))
'''
#Sara Nassar - sana09516
def calc_entropy_from_probabilities(list_probas):
    entropy_value = 0

    for proba in list_probas:
        # If the probability is not zero
        if proba != 0:
            entropy_value += -proba * np.log(proba)
     
    return entropy_value

In [139]:
# Test your function
list_probbs=[1/5, 2/5, 2/5]
print(calc_entropy_from_probabilities(list_probbs))

1.0549201679861442


In [140]:
def information_gain(old_entropy,new_entropies,count_items):
    '''
    Author: Sara Nassar 
    from the list of new entropies, calculate the overall new entropy
    
    formula is something like:
    overall_new_entropy = entropy1*proportion1 + entropy2*proportion2+ entropy3*proportion3 ...
    
    igain=old_entropy-overall_new_entropy
    '''
    
    overall_new_entropy = 0
    
    # Calculating the total number of items
    total_items = sum(count_items)
    
    for i in range(len(new_entropies)):
        # Calculating the proportion of items in the current partition
        proportion = count_items[i] / total_items
        
        # Adding the entropy of the current partition weighted by its proportion to the overall new entropy
        overall_new_entropy += new_entropies[i] * proportion
        
    # Calculating the information gain
    information_gain = old_entropy - overall_new_entropy
    
    return information_gain

In [141]:
num_feats=X_test.shape[1]
def initialize_weights(number_features):
    '''
    the first set of weights corresponding to the features
    For now, it defaults to 2
    '''
    
    weights=np.array([2 for i in range(number_features)])
    return weights

In [142]:
print(initialize_weights(num_feats))

[2 2 2 2]


In [143]:
y_test

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0,
       0, 1, 2, 2, 1, 2])

In [144]:
# initialization
# Step 1: Calculate the probabilities of 0, 1 and 2 in the y_test array
'''
Author: Sara Nassar 
'''
proba_init=calculate_probabilities(y_test,uniq_labels)#get the probabilities for y_test
print("Initial proba=",proba_init)


Initial proba= [0.38, 0.3, 0.32]


In [145]:
# Step 2: Calculate the initial entropy of y_test, using the probability values
'''
Author: Sara Nassar 
'''
# you might have to convert the dictionary to a list
# get only the probability values
list_probas=proba_init#get list from dictionary proba_init
print(list_probas)

entropy_init=calc_entropy_from_probabilities(list_probas)# call the entropy calculation function using the list probabilities
print("Initial entropy = ",entropy_init)

[0.38, 0.3, 0.32]
Initial entropy =  1.0934927418975058


In [146]:
wt_init=initialize_weights(num_feats)
# right now the initialize_weights function only returns 2,2,2 
print(wt_init)

# multiply the weights with each feature and calculate the sum
#res=# use np.sum() function
#print(res)

'''
Author: Sara Nassar 
'''

def calculate_sum(weights, features):
    # Make sure the shapes of the arrays match
    if weights.shape[0] != features.shape[1]:
        raise ValueError("The number of weights must be equal to the number of columns in the features matrix.")

    # Calculate the dot product
    total_sum = np.dot(features, weights)
    
    return total_sum

# multiply the weights with each feature and calculate the sum
# Call the function
res = calculate_sum(wt_init, X_test)

# Print the result
print(res)

[2 2 2 2]
[29.6 23.  39.  29.8 31.6 21.4 26.8 34.8 28.8 27.2 33.6 18.6 21.  19.2
 21.4 31.8 35.  26.2 28.6 34.  19.4 31.6 20.8 33.8 40.2 34.4 33.6 36.4
 19.  19.4 18.8 24.  31.2 20.  18.2 31.4 31.2 20.8 20.4 21.8 31.  31.
 32.  22.  21.6 25.2 31.4 33.6 30.8 38.8]


In [147]:

'''
Author: Sara Nassar 
'''
def choose_threshold(features):
    # Get the minimum and maximum values from the features matrix
    min_value = np.min(features)
    max_value = np.max(features)
    
    # Generate a random threshold between the minimum and maximum values
    threshold = np.random.uniform(min_value, max_value)
    
    return threshold

# Call the function
threshold = choose_threshold(res) 

# Print the result
print(threshold)

29.566555045240882
